In [1]:
import json
from py2neo import Graph, Node, Relationship

In [2]:
# 连接到 Neo4j 数据库
graph = Graph("bolt://localhost:7687", auth=("neo4j", "12345678"))

In [3]:
# Define function to create nodes and relationships
def create_graph(graph, data):
    for entry in data:
        nctid = entry.get("NCTID")
        title = entry.get("Study_Title")
        sponsor = entry.get("Sponsor")
        collaborators = entry.get("Collaborators")
        conditions = entry.get("Conditions")
        brief_summary = entry.get("BriefSummary")
        inclusion_criteria = entry.get("InclusionCriteria")
        exclusion_criteria = entry.get("ExclusionCriteria")
        interventions = entry.get("Interventions")

        # 创建 NCTid 节点
        nctid_node = Node("NCTid", id=nctid, title=title, brief=brief_summary,
                          inclusion_criteria=inclusion_criteria, exclusion_criteria=exclusion_criteria)
        graph.merge(nctid_node, "NCTid", "id")

        # 创建 Sponsor 节点和关系
        if str(sponsor) != 'nan':
            sponsor_node = Node("Sponsor", name=sponsor)
            graph.merge(sponsor_node, "Sponsor", "name")
            graph.merge(Relationship(sponsor_node, "SPONSORS", nctid_node))

        # 创建 Collaborators 节点和关系
        if collaborators:
            for collaborator in collaborators:
                # 确保 collaborator 不为空
                collaborator_node = Node("Collaborator", name=collaborator)
                graph.merge(collaborator_node, "Collaborator", "name")
                graph.merge(Relationship(collaborator_node, "COLLABORATES_WITH", nctid_node))

        # # 创建 Conditions 节点和关系
        if conditions:
            for condition in conditions:
                if condition:  # 确保 condition 不为空
                    condition_node = Node("Condition", name=condition)
                    graph.merge(condition_node, "Condition", "name")
                    graph.merge(Relationship(nctid_node, "HAS_CONDITION", condition_node))
        # 
        # # 创建 Interventions 节点和关系
        if interventions:
            for intervention_type, items in interventions.items():
                for item in items:
                    if str(item)!='nan':  # 确保 item 不为空
                        intervention_node = Node(intervention_type.capitalize(), name=item)
                        graph.merge(intervention_node, intervention_type.capitalize(), "name")
                        graph.merge(Relationship(nctid_node, "HAS_INTERVENTION"+intervention_type.capitalize(), intervention_node))


# 读取 JSON 文件
with open('图谱2.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 创建节点和关系在 Neo4j 中
create_graph(graph, data)

print("知识图谱创建成功。")

produce intervention_type Produce
drugs intervention_type Drugs
produce intervention_type Produce
produce intervention_type Produce
produce intervention_type Produce
produce intervention_type Produce
produce intervention_type Produce
produce intervention_type Produce
drugs intervention_type Drugs
other intervention_type Other
drugs intervention_type Drugs
drugs intervention_type Drugs
produce intervention_type Produce
produce intervention_type Produce
other intervention_type Other
other intervention_type Other
drugs intervention_type Drugs
drugs intervention_type Drugs
drugs intervention_type Drugs
drugs intervention_type Drugs
drugs intervention_type Drugs
other intervention_type Other
other intervention_type Other
devices intervention_type Devices
drugs intervention_type Drugs
drugs intervention_type Drugs
other intervention_type Other
other intervention_type Other
produce intervention_type Produce
produce intervention_type Produce
produce intervention_type Produce
devices interventi

In [ ]:
from py2neo import Graph

graph = Graph("bolt://localhost:7687", auth=("neo4j", "12345678"))

# 查询所有 NCTid 节点及其关系
query = """
MATCH (n:NCTid)-[r]->(m)
RETURN n.id AS nctid, type(r) AS relation, m.name AS entity
"""
results = graph.run(query)

# 将三元组保存到列表
triples = []
for result in results:
    nctid = result["nctid"]
    relation = result["relation"]
    entity = result["entity"]
    triples.append((nctid, relation, entity))

# 保存三元组到文件
with open("triples.txt", "w", encoding="utf-8") as f:
    for triple in triples:
        f.write(f"{triple[0]}\t{triple[1]}\t{triple[2]}\n")
